# Open-Sora 2.0 Training

## モデルの訓練

訓練は3つのステージで構成:

1. 低解像度動画によるT2V（Text to Video）モデルの訓練
1. 低解像度動画によるI2V（Image to Video）モデルの訓練
1. 高解像度動画によるI2Vモデルのファインチューニング

訓練のコストの内訳は、ステージ1のコストが50%を占める:

![](image/table3.png)

訓練の効率化の戦略:

- 11B（110億）パラメータのFluxの蒸留モデルで、T2Vモデルを初期化
    - Fluxは、オープンソースT2I（Text to Image）モデル
- 高品質な動画データで、訓練効率を向上
    - PixArtから着想
    - ステップ1, 2では、大規模データセットから高品質なサブセットを抽出
    - ステップ3では、より厳しいフィルタリングでサブセットを抽出
- 低解像度動画での動きの学習
    - 高解像度動画の訓練コストは高いため、多様な動きは低解像度動画を中心に訓練
        - 128フレーム768pxの動画の訓練は、256pxの場合よりも40倍遅い
        - セルフアテンションの計算複雑度が二次関数的に増大するため
    - 出力がぼやけるため、高解像度動画の訓練で品質を向上
- ステップ2のアップスケーリング学習は、T2VではなくI2Vを採用し効率化:
    - 静止画を条件付けすることで、動きの生成に集中できるため
- 高解像度動画でのファインチューニングを短縮し、訓練を効率化

Open-Sora 2.0の訓練コストは、5~10倍低い:

![](image/fig7.png)

訓練設定は、Open-Sora 1.2に基づく:

- 目的関数はFlow Matchingを採用
- オプティマイザはAdamWを使用
    - $\beta$値は$(0.9, 0.999)$
    - $\epsilon$は$1\times 10^{-15}$
    - 重み減衰は使用しない
    - 学習率
        - ステージ1, 2
            - 最初の4万ステップは$5\times 10^{-5}$
            - 最後の4.5万ステップは$3\times 10^{-5}$
        - ステージ3
            - $1\times 10^{-5}$
    - 購買ノルムクリッピングの閾値は$1$

Flow matchingは、Stable Diffusion 3（SD3）と類似:

$$
\mathcal{L} = \mathbb{E}_{t,X_{0},X_{1}}[||f_{\theta}(X_{t},t,y)-(X_{0}-X_{1})||]
$$

- $X_0$: 動画の潜在表現
- $X_1 \sim \mathcal{N}(0,1)$: ガウスノイズ
- $t$: タイムステップ（ノイズ強度）
- $X_t=(1-t)X_0 + tX_1$: 補間された潜在表現（ノイズありの潜在表現）
- $y$: テキストや画像などの条件
- $(X_0 - X_1)$: 正解の速度（データからノイズへの変化の方向）
- $\mathbb{E}_{t, X_0, X_1}$: 条件を変えた時の平均

$t$は、対数正規分布からサンプリングし、$X_0$の形状に合わせてスケーリング：

$$
t' = \frac{at}{1 + (a-1)t}
$$

- $a$: $T\times H\times W$に比例するパラメータ
- 高解像度での長時間の画像はノイズの影響を受けやすいため
- 推論時も同様の方法が適用

マルチバケットトレーニングを採用:

- フレーム数・解像度・アスペクト比が似た動画をバケツに分けて、バケツごとにバッチサイズを調整する手法
    - メモリ不足（OOM）の回避
    - 並列計算時の訓練時間の均一化

ステージ1, 2のバッチサイズとバッチサイズとスループット:

![](image/table4.png)

ステージ3のバッチサイズとスループット（Context parallelism 4）:

![](image/table5.png)

高圧縮オートエンコーダー（DC-AE）を使用し、トレーニングコストをさらに削減

DC-AEを使用すると、動画生成に必要なトークン数が大幅に削減できる:

$$
D_{token} = D_{T} \times D_{H} \times D_{W} \times P_{T} \times P_{H} \times P_{W}
$$

- $D_{\text{token}}$: トークン・ダウンサンプル比（どのくらい小さく圧縮してトークン化するかの比率）
- $D_T, D_H, H_W$: 時間・高さ・幅の次元におけるオートエンコーダの圧縮率
- $P_T, P_H, P_W$: 生成モデルに入力する際のパッチ分割サイズ
- Hunyuan Videoの$D_{\text{token}}$は$4096$、DC-AEの$D_{\text{token}}$は$1024$

オートエンコーダーの性能は、情報ダウンサンプリング比で予測できる:

$$
D_{info} = \frac{D_{T} \times D_{H} \times D_{W} \times C_{in}}{C_{out}}
$$

- $D_{\text{info}}$: 入力データが潜在表現になるときに情報量が圧縮された比率
- $C_{\text{in}}$: 入力チャンネル数（RGBの場合$3$）
- $C_{\text{out}}$: 出力チャンネル数（潜在表現のチャンネル数）
- DC-AEのオートエンコーダは、Hunyuan Video VAEとStepVideo VAEの$D_{\text{info}}$と一致するように設計
- 具体的には出力チャンネル数を増やす

DC-AEのダウンサンプルブロックとアップサンプルブロックには、勾配伝播の問題がある:

- ピクセルシャッフルとピクセルアンシャッフルを使用した残差接続を導入

DC-AEで出力した高圧縮の潜在変数でモデルを訓練すると、動画生成の品質が低くなる:

- チャンネル数が多いと、潜在空間構造（latent space structure）の最適化が難しくなるため
- DC-AE訓練後に、第3層の潜在表現をDINOv2に合わせるための蒸留損失を適用し、最適化

DC-AEを使用した動画生成モデルの訓練:

1. 2000万件の最大33フレームの動画を、1.7万ステップで訓練
2. 200万件の最大128フレームの動画を、8000ステップで訓練

DC-AEを使用した動画生成モデルは、Hunyuan Video VAEよりも、訓練と推論で高い効率性:

![](image/fig8.png)

DC-AEは、256チャネルと128チャネルの2つを最初から訓練:

1. 再構成損失$\mathcal{L}_1$と知覚損失$\mathcal{L}_{\text{LIPIS}}$を使用して、25万ステップ訓練:
    $$
    \mathcal{L} = \mathcal{L}_{1} + 0.5\mathcal{L}_{LPIPS}
    $$
2. 敵対的損失$\mathcal{L}_{\text{adv}}$を使用して、20万ステップ訓練:
    $$
    \mathcal{L} = \mathcal{L_1} + 0.5\mathcal{L_{LPIPS}} + 0.05\mathcal{L_{\text{adv}}}
    $$

DC-AEの訓練設定:

- ローカルバッチサイズ$1$で$8$個のGPUを使用
- アスペクト比$1:1$、32フレーム、256pxの動画を使用
- オプティマイザはAdamWを使用（$\beta=(0.9, 0.999)$, $\epsilon=1\times 10^{-15}$, 重み減衰なし）
- オートエンコーダの学習率は5e-5
- ディスクリミネータの学習率は1e-4
- 勾配ノルムクリップは閾値1

## 条件付け

条件付けは、潜在変数に対して条件ベクトルとタスクの種類を示すチャンネルを連結して行う:

![](image/fig10.png)

- 左: 画像から動画を生成するタスク
- 中: 動画を延長するタスク
- 右: 最初と最後の画像の間を補完するタスク
- 連結後のチャンネル数は$2k + 1$

汎化性能を向上させるために、テキスト条件のドロップアウトや画像条件のドロップアウトを導入

推論には、Classifier-free Guidance（CFG）を採用:

$$
v_{t} = v_{\theta}(x_{t},t,\emptyset,\emptyset) + g_{img}\cdot(v_{\theta}(x_{t},t,\emptyset,img)-v_{\theta}(x_{t},t,\emptyset,\emptyset)) + g_{txt}\cdot(v_{\theta}(x_{t},t,txt,img)-v_{\theta}(x_{t},t,\emptyset,img))
$$

- $g_{\text{img}}$: 画像条件の強さを調整する係数
- $g_{\text{txt}}$: テキスト条件の強さを調整する係数

画像ガイダンスを強くすると、生成された動画にチラつき（flicker）が生じる:

- Guidance Oscillationを導入して安定化
    - 例えば、50ステップのサンプリングのうち、10ステップごとに$g_{\text{img}}$を$1$に落とす

I2Vでは動的な画像ガイダンス（dynamic image guidance）を採用し、動画の後半になるにつれて条件付けを強くする:

![](image/fig11.png)

- 横軸はフレーム数
- 縦軸は推論ステップ数
- 画像のガイダンススケールは$1$から$k$まで線形にスケーリング
- 推論が終わりになるにつれて、ガイダンススケーリングが$1$になるように減衰
- ガイダンスのデフォルト値は$g_{\text{img}}=3$, $g_{\text{txt}}=7.5$

動画の動きの強さはモーションスコアをキャプションに追加することで制御:

![](image/fig12.png)

## システム最適化

訓練は、ColossalAIを使用:

- 141GBのメモリ容量を持つH200 GPUを使用し、データ並列（DP, Data Parallelism）を導入
- 選択的アクティベーションチェックポインティング（Selective Activation Checkpointing）を導入
    - 一部の順伝播の計算結果を破棄し、必要に応じて再計算する技術でメモリ効率化


高解像度の動画を効率的に処理するために、複数の並列化技術を採用:

- AEでは、テンソル並列化（TP, Tensor Parallelism）を畳み込み層に適用
    - 入力または出力のチャネル次元で重みを分割する
- MMDiTでは、Zero Redundancy Opitimizer（ZeroDP）とコンテキスト並列（CP, Context Parallelism）を適用
    - ZeroDPは、DeepSpeedのZeRO2により勾配を分割し、重複なくGPUが保持する技術
    - CPは、動画とテキストのシーケンスを分割し、各GPUが独立してアテンション計算する技術

アクティベーションチェックポインティングは、ブロックの入力のみを保持し、それ以外は再計算する:

- ステージ1, 2では、デュアルブロックの8層とすべてのシングルブロックに適用
- ステージ3では、すべてのブロックで有効化し、CPUへのオフロードも使用
    - オフロードでは、Pinned Memoryと非同期データ転送を使用し、さらに効率化

分散訓練の場合、物理障害が発生しやすいので自動復旧機能（Auto Recovery）を導入

- InfiniBandの障害、ストレージシステムのクラッシュ、NCCLエラー
- 訓練の状態を監視し、速度低下・損失の急激な悪化・レスポンスがないかを検証
- 問題が検出された場合は、すべてのプロセスを停止し、ノードを診断
- 必要に応じて予備をデプロイし、最終チェックポイントから自動的に訓練を再開する
- GPU稼働率99%で、ダウンタイムを最小限に抑えた

CPUとGPU間のデータ転送を高速化するためにPyTorchのデータローダーを最適化:

- PyTorchデフォルトのPinned Memoryは、CUDAカーネル実行をブロックする可能性がある
- メモリバッファ（pre-allocated pinned memory buffer）を使用し、オーバーヘッドを削減
- データ転送と計算をオーバーラップさせて効率化
- グローバルなGC（ガベージコレクション）は、分散訓練ではバグが多いため手動でメモリかんり

チェックポイントの保存は、ピン留めされたメモリバッファに対して直接アクセスすることで高速化:

- ディスクへの書き込みは、C++による非同期ディスク書き込みにより訓練プロセスをブロックせずに効率化

## 実装

In [ ]:
try:
    import google.colab
    IN_COLAB = True
    !git clone https://github.com/hpcaitech/Open-Sora.git
    WORK_DIR = "/content/Open-Sora"
    %cd $WORK_DIR
except:
    IN_COLAB = False
    WORK_DIR = "/workspaces/open-sora/Open-Sora"
    %cd $WORK_DIR

In [ ]:
import logging
import os
import platform
import subprocess

DEBUG_LOG_PATH = os.path.join(WORK_DIR, "debug.log")

if os.path.exists(DEBUG_LOG_PATH):
    os.remove(DEBUG_LOG_PATH)

def custom_format(record):
    match record.levelno:
        case logging.DEBUG:
            level = "🟦"
        case logging.INFO:
            level = "🟩"
        case logging.WARNING:
            level = "🟨"
        case logging.ERROR:
            level = "🟥"
        case logging.CRITICAL:
            level = "🛑"
    return f"{level} {record.getMessage()}"

logger = logging.getLogger()

for handler in logger.handlers:
    logger.removeHandler(handler)

formatter = logging.Formatter()
formatter.format = custom_format

file_handler = logging.FileHandler(DEBUG_LOG_PATH)
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.setLevel(logging.DEBUG)

PYTHON_VERSION = platform.python_version()
logger.info(f"Python {PYTHON_VERSION}")

NVIDIA_SMI = subprocess.run("nvidia-smi", capture_output=True, text=True).stdout
logger.info(f"{NVIDIA_SMI}")

In [ ]:
if IN_COLAB:
    %pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124

    %pip install \
        accelerate \
        av==13.1.0 \
        colossalai \
        ftfy \
        liger-kernel \
        omegaconf \
        mmengine \
        openai \
        pandas \
        pandarallel \
        pyarrow \
        tensorboard \
        wandb \
        --extra-index-url https://download.pytorch.org/whl/cu124

    %pip install flash-attn --no-build-isolation

    %pip install -e . --no-deps

In [ ]:
from dataclasses import dataclass, field
from einops import rearrange
from flash_attn import flash_attn_func as flash_attn_func_v2
from functools import partial
from huggingface_hub import PyTorchModelHubMixin
from inspect import signature
from liger_kernel.ops.rms_norm import LigerRMSNormFunction
from liger_kernel.ops.rope import LigerRopeFunction
from omegaconf import MISSING, OmegaConf
from torch import Tensor, nn
from torch.nn.modules.batchnorm import _BatchNorm
from typing import Any, Callable, Optional, Union, Tuple
import diffusers
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import argparse
from mmengine.config import Config
import ast

try:
    from flash_attn_interface import flash_attn_func as flash_attn_func_v3
    SUPPORT_FA3 = True
except:
    SUPPORT_FA3 = False

## データセット

In [ ]:
DATASETS_DIR = os.path.join(WORK_DIR, "datasets")
os.makedirs(DATASETS_DIR, exist_ok=True)

PEXELS_45K_NECESSARY_PATH = os.path.join(DATASETS_DIR, "pexels_45k_necessary.csv")
PEXELS_45K_SCORE_PATH = os.path.join(DATASETS_DIR, "pexels_45k_score.csv")

In [ ]:
import pandas as pd
pexels_45k_score_df = pd.read_csv(PEXELS_45K_SCORE_PATH)
pexels_45k_score_df.head()

# sort by duration
pexels_45k_score_df = pexels_45k_score_df.sort_values(by=["filesize"])
pexels_45k_score_df.head()

In [ ]:
import pandas as pd
pexels_45k_necessary = pd.read_csv(PEXELS_45K_NECESSARY_PATH)

# Take first 10 samples and save as csv
pexels_45k_necessary.sort_values(by=["num_frames"]).head(10).to_csv("pexels_45k_necessary_top10.csv", index=False)

## CLIパーサ

In [ ]:
def parse_args(args) -> tuple[str, argparse.Namespace]:
    """
    This function parses the command line arguments.

    Returns:
        tuple[str, argparse.Namespace]: The path to the configuration file and the command line arguments.
    """
    parser = argparse.ArgumentParser()
    parser.add_argument("config", type=str, help="model config file path")
    args, unknown_args = parser.parse_known_args(args)
    return args.config, unknown_args

In [ ]:
def read_config(config_path: str) -> Config:
    """
    This function reads the configuration file.

    Args:
        config_path (str): The path to the configuration file.

    Returns:
        Config: The configuration object.
    """
    cfg = Config.fromfile(config_path)
    return cfg

In [ ]:
def auto_convert(value: str) -> int | float | bool | list | dict | None:
    """
    Automatically convert a string to the appropriate Python data type,
    including int, float, bool, list, dict, etc.

    Args:
        value (str): The string to convert.

    Returns:
        int, float, bool, list |  dict: The converted value.
    """
    # Handle empty string
    if value == "":
        return value

    # Handle None
    if value.lower() == "none":
        return None

    # Handle boolean values
    lower_value = value.lower()
    if lower_value == "true":
        return True
    elif lower_value == "false":
        return False

    # Try to convert the string to an integer or float
    try:
        # Try converting to an integer
        return int(value)
    except ValueError:
        pass

    try:
        # Try converting to a float
        return float(value)
    except ValueError:
        pass

    # Try to convert the string to a list, dict, tuple, etc.
    try:
        return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        pass

    # If all attempts fail, return the original string
    return value


In [ ]:
def merge_args(cfg: Config, args: argparse.Namespace) -> Config:
    """
    This function merges the configuration file and command line arguments.

    Args:
        cfg (Config): The configuration object.
        args (argparse.Namespace): The command line arguments.

    Returns:
        Config: The configuration object.
    """
    for k, v in zip(args[::2], args[1::2]):
        assert k.startswith("--"), f"Invalid argument: {k}"
        k = k[2:].replace("-", "_")
        k_split = k.split(".")
        target = cfg
        for key in k_split[:-1]:
            assert key in cfg, f"Key {key} not found in config"
            target = target[key]
        if v.lower() == "none":
            v = None
        elif k in target:
            v_type = type(target[k])
            if v_type == bool:
                v = auto_convert(v)
            else:
                v = type(target[k])(v)
        else:
            v = auto_convert(v)
        target[k_split[-1]] = v
    return cfg

In [ ]:
def parse_configs(args) -> Config:
    """
    This function parses the configuration file and command line arguments.

    Returns:
        Config: The configuration object.
    """
    config, args = parse_args(args)
    cfg = read_config(config)
    cfg = merge_args(cfg, args)
    cfg.config_path = config

    # hard-coded for spatial compression
    if cfg.get("ae_spatial_compression", None) is not None:
        os.environ["AE_SPATIAL_COMPRESSION"] = str(cfg.ae_spatial_compression)

    return cfg

## DC-AEの訓練

In [ ]:
# DC-AE訓練

# https://github.com/hpcaitech/Open-Sora/blob/main/docs/hcae.md

# torchrun --nproc_per_node 8 scripts/vae/train.py configs/vae/train/video_dc_ae.py

dcae_cfg = parse_configs([
    "configs/vae/train/video_dc_ae.py",
])
dcae_cfg

In [ ]:
import gc
import os
import random
import subprocess
import warnings
from contextlib import nullcontext
from copy import deepcopy
from pprint import pformat

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
gc.disable()


import torch
import torch.distributed as dist
from colossalai.booster import Booster
from colossalai.utils import set_seed
from torch.profiler import ProfilerActivity, profile, schedule
from tqdm import tqdm

import wandb
from opensora.acceleration.checkpoint import set_grad_checkpoint
from opensora.acceleration.parallel_states import get_data_parallel_group
from opensora.datasets.dataloader import prepare_dataloader
from opensora.datasets.pin_memory_cache import PinMemoryCache
from opensora.models.vae.losses import DiscriminatorLoss, GeneratorLoss, VAELoss
from opensora.registry import DATASETS, MODELS, build_module
from opensora.utils.ckpt import CheckpointIO, model_sharding, record_model_param_shape, rm_checkpoints
# from opensora.utils.config import config_to_name, create_experiment_workspace, parse_configs
from opensora.utils.config import config_to_name, create_experiment_workspace
from opensora.utils.logger import create_logger
from opensora.utils.misc import (
    Timer,
    all_reduce_sum,
    create_tensorboard_writer,
    is_log_process,
    log_model_params,
    to_torch_dtype,
)
from opensora.utils.optimizer import create_lr_scheduler, create_optimizer
from opensora.utils.train import create_colossalai_plugin, set_lr, set_warmup_steps, setup_device, update_ema

In [ ]:
torch.backends.cudnn.benchmark = True

WAIT = 1
WARMUP = 10
ACTIVE = 20

my_schedule = schedule(
    wait=WAIT,  # number of warmup steps
    warmup=WARMUP,  # number of warmup steps with profiling
    active=ACTIVE,  # number of active steps with profiling
)

In [ ]:
# ======================================================
# 1. configs & runtime variables
# ======================================================
# == parse configs ==
# cfg = parse_configs()
cfg = parse_configs([
    "configs/vae/train/video_dc_ae.py",
])

cfg.bucket_config = {
    "256px_ar1:1": {
        8: (1.0, 1),
        32: (0.0, 1),
    }
}
cfg.epochs = 1

In [ ]:
# == get dtype & device ==
dtype = to_torch_dtype(cfg.get("dtype", "bf16"))

os.environ['RANK'] = '0'
os.environ['WORLD_SIZE'] = '1'
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12345'

device, coordinator = setup_device()
checkpoint_io = CheckpointIO()
set_seed(cfg.get("seed", 1024))
PinMemoryCache.force_dtype = dtype
pin_memory_cache_pre_alloc_numels = cfg.get("pin_memory_cache_pre_alloc_numels", None)
PinMemoryCache.pre_alloc_numels = pin_memory_cache_pre_alloc_numels

In [ ]:
# == init ColossalAI booster ==
plugin_type = cfg.get("plugin", "zero2")
plugin_config = cfg.get("plugin_config", {})
plugin = (
    create_colossalai_plugin(
        plugin=plugin_type,
        dtype=cfg.get("dtype", "bf16"),
        grad_clip=cfg.get("grad_clip", 0),
        **plugin_config,
    )
    if plugin_type != "none"
    else None
)
booster = Booster(plugin=plugin)

In [ ]:
# == init exp_dir ==
exp_name, exp_dir = create_experiment_workspace(
    cfg.get("outputs", "./outputs"),
    model_name=config_to_name(cfg),
    config=cfg.to_dict(),
)
if is_log_process(plugin_type, plugin_config):
    print(f"changing {exp_dir} to share")
    os.system(f"chgrp -R share {exp_dir}")

In [ ]:
# == init logger, tensorboard & wandb ==
logger = create_logger(exp_dir)
logger.info("Training configuration:\n %s", pformat(cfg.to_dict()))
tb_writer = None
if coordinator.is_master():
    tb_writer = create_tensorboard_writer(exp_dir)
    if cfg.get("wandb", False):
        wandb.init(
            project=cfg.get("wandb_project", "Open-Sora"),
            name=cfg.get("wandb_expr_name", exp_name),
            config=cfg.to_dict(),
            dir=exp_dir,
        )

In [ ]:
# ======================================================
# 2. build dataset and dataloader
# ======================================================
logger.info(f"Building dataset... {cfg.dataset=}")

In [ ]:
# TODO: pexels_45k_necessary.csvをダウンロード

assert os.path.join(WORK_DIR, cfg.dataset.data_path)

In [ ]:
DATASETS

In [ ]:
# == build dataset ==

# VideoTextDatasetを実体化
dataset = build_module(cfg.dataset, DATASETS)
logger.info("Dataset contains %s samples.", len(dataset))

In [ ]:
# == build dataloader ==
cache_pin_memory = pin_memory_cache_pre_alloc_numels is not None
dataloader_args = dict(
    dataset=dataset,
    batch_size=cfg.get("batch_size", None),
    num_workers=cfg.get("num_workers", 4),
    seed=cfg.get("seed", 1024),
    shuffle=True,
    drop_last=True,
    pin_memory=True,
    process_group=get_data_parallel_group(),
    prefetch_factor=cfg.get("prefetch_factor", None),
    cache_pin_memory=cache_pin_memory,
)
dataloader, sampler = prepare_dataloader(
    bucket_config=cfg.get("bucket_config", None),
    num_bucket_build_workers=cfg.get("num_bucket_build_workers", 1),
    **dataloader_args,
)
num_steps_per_epoch = len(dataloader)

In [ ]:
first_batch = next(iter(dataloader))
first_batch["sampling_interval"], first_batch["video"].shape, first_batch["path"], first_batch["index"]

In [ ]:
first_frame = first_batch["video"][:, :, 0].to(device=device, dtype=torch.float32)
first_frame.shape

import matplotlib.pyplot as plt
plt.imshow(first_frame[0].permute(1, 2, 0).cpu().numpy())
plt.axis("off")
plt.show()

In [ ]:
# ======================================================
# 3. build model
# ======================================================
logger.info("Building models...")

# == build vae model ==
model = build_module(cfg.model, MODELS, device_map=device, torch_dtype=dtype).train()
log_model_params(model)
model

In [ ]:
if cfg.get("grad_checkpoint", False):
    set_grad_checkpoint(model)

vae_loss_fn = VAELoss(**cfg.vae_loss_config, device=device, dtype=dtype)

In [ ]:
# == build EMA model ==
if cfg.get("ema_decay", None) is not None:
    ema = deepcopy(model).cpu().eval().requires_grad_(False)
    ema_shape_dict = record_model_param_shape(ema)
    logger.info("EMA model created.")
else:
    ema = ema_shape_dict = None
    logger.info("No EMA model created.")

In [ ]:
# == build discriminator model ==
use_discriminator = cfg.get("discriminator", None) is not None
if use_discriminator:
    discriminator = build_module(cfg.discriminator, MODELS).to(device, dtype).train()
    log_model_params(discriminator)
    generator_loss_fn = GeneratorLoss(**cfg.gen_loss_config)
    discriminator_loss_fn = DiscriminatorLoss(**cfg.disc_loss_config)

In [ ]:
# == setup optimizer ==
optimizer = create_optimizer(model, cfg.optim)

In [ ]:
# == setup lr scheduler ==
lr_scheduler = create_lr_scheduler(
    optimizer=optimizer, num_steps_per_epoch=num_steps_per_epoch, epochs=cfg.get("epochs", 1000), **cfg.lr_scheduler
)

In [ ]:
# == setup discriminator optimizer ==
if use_discriminator:
    disc_optimizer = create_optimizer(discriminator, cfg.optim_discriminator)
    disc_lr_scheduler = create_lr_scheduler(
        optimizer=disc_optimizer,
        num_steps_per_epoch=num_steps_per_epoch,
        epochs=cfg.get("epochs", 1000),
        **cfg.disc_lr_scheduler,
    )

In [ ]:
# =======================================================
# 4. distributed training preparation with colossalai
# =======================================================
logger.info("Preparing for distributed training...")
# == boosting ==
torch.set_default_dtype(dtype)
model, optimizer, _, dataloader, lr_scheduler = booster.boost(
    model=model,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler,
    dataloader=dataloader,
)

In [ ]:
if use_discriminator:
    discriminator, disc_optimizer, _, _, disc_lr_scheduler = booster.boost(
        model=discriminator,
        optimizer=disc_optimizer,
        lr_scheduler=disc_lr_scheduler,
    )

In [ ]:
torch.set_default_dtype(torch.float)
logger.info("Boosted model for distributed training")

In [ ]:
# == global variables ==
cfg_epochs = cfg.get("epochs", 1000)
mixed_strategy = cfg.get("mixed_strategy", None)
mixed_image_ratio = cfg.get("mixed_image_ratio", 0.0)
# modulate mixed image ratio since we force rank 0 to be video
num_ranks = dist.get_world_size()
modulated_mixed_image_ratio = (
    num_ranks * mixed_image_ratio / (num_ranks - 1) if num_ranks > 1 else mixed_image_ratio
)
if is_log_process(plugin_type, plugin_config):
    print("modulated mixed image ratio:", modulated_mixed_image_ratio)

start_epoch = start_step = log_step = acc_step = 0
running_loss = dict(  # loss accumulated over config.log_every steps
    all=0.0,
    nll=0.0,
    nll_rec=0.0,
    nll_per=0.0,
    kl=0.0,
    gen=0.0,
    gen_w=0.0,
    disc=0.0,
    debug=0.0,
)

def log_loss(name, loss, loss_dict, use_video):
    # only calculate loss for video
    if use_video == 0:
        loss.data = torch.tensor(0.0, device=device, dtype=dtype)
    all_reduce_sum(loss.data)
    num_video = torch.tensor(use_video, device=device, dtype=dtype)
    all_reduce_sum(num_video)
    loss_item = loss.item() / num_video.item()
    loss_dict[name] = loss_item
    running_loss[name] += loss_item

logger.info("Training for %s epochs with %s steps per epoch", cfg_epochs, num_steps_per_epoch)

In [ ]:
# == sharding EMA model ==
if ema is not None:
    model_sharding(ema)
    ema = ema.to(device)

if cfg.get("freeze_layers", None) == "all":
    for param in model.module.parameters():
        param.requires_grad = False
    print("all layers frozen")

# model.module.requires_grad_(False)

In [ ]:
# == resume ==
if cfg.get("load", None) is not None:
    logger.info("Loading checkpoint from %s", cfg.load)
    start_epoch = cfg.get("start_epoch", None)
    start_step = cfg.get("start_step", None)
    ret = checkpoint_io.load(
        booster,
        cfg.load,
        model=model,
        ema=ema,
        optimizer=optimizer,
        lr_scheduler=lr_scheduler,
        sampler=(
            None if start_step is not None else sampler
        ),  # if specify start step, set last_micro_batch_access_index of a new sampler instead
    )
    if start_step is not None:
        # if start step exceeds data length, go to next epoch
        if start_step > num_steps_per_epoch:
            start_epoch = (
                start_epoch + start_step // num_steps_per_epoch
                if start_epoch is not None
                else start_step // num_steps_per_epoch
            )
            start_step = start_step % num_steps_per_epoch
        sampler.set_step(start_step)

    start_epoch = start_epoch if start_epoch is not None else ret[0]
    start_step = start_step if start_step is not None else ret[1]

    if (
        use_discriminator
        and os.path.exists(os.path.join(cfg.load, "discriminator"))
        and not cfg.get("restart_disc", False)
    ):
        booster.load_model(discriminator, os.path.join(cfg.load, "discriminator"))
        if cfg.get("load_optimizer", True):
            booster.load_optimizer(disc_optimizer, os.path.join(cfg.load, "disc_optimizer"))
            if disc_lr_scheduler is not None:
                booster.load_lr_scheduler(disc_lr_scheduler, os.path.join(cfg.load, "disc_lr_scheduler"))
            if cfg.get("disc_lr", None) is not None:
                set_lr(disc_optimizer, disc_lr_scheduler, cfg.disc_lr)

    logger.info("Loaded checkpoint %s at epoch %s step %s", cfg.load, start_epoch, start_step)

    if cfg.get("lr", None) is not None:
        set_lr(optimizer, lr_scheduler, cfg.lr, cfg.get("initial_lr", None))

    if cfg.get("update_warmup_steps", False):
        assert (
            cfg.lr_scheduler.get("warmup_steps", None) is not None
        ), "you need to set lr_scheduler.warmup_steps in order to pass --update-warmup-steps True"
        set_warmup_steps(lr_scheduler, cfg.lr_scheduler.warmup_steps)
        if use_discriminator:
            assert (
                cfg.disc_lr_scheduler.get("warmup_steps", None) is not None
            ), "you need to set disc_lr_scheduler.warmup_steps in order to pass --update-warmup-steps True"
            set_warmup_steps(disc_lr_scheduler, cfg.disc_lr_scheduler.warmup_steps)

In [ ]:
# =======================================================
# 5. training loop
# =======================================================
dist.barrier()

cfg["accumulation_steps"] = 128
accumulation_steps = int(cfg.get("accumulation_steps", 1))
logger.info("Using gradient accumulation steps: %s", accumulation_steps)

In [ ]:

for epoch in range(start_epoch, cfg_epochs):
    # == set dataloader to new epoch ==
    sampler.set_epoch(epoch)
    dataiter = iter(dataloader)
    logger.info("Beginning epoch %s...", epoch)
    random.seed(1024 + dist.get_rank())  # load vid/img for each rank

    # == training loop in an epoch ==
    with tqdm(
        enumerate(dataiter, start=start_step),
        desc=f"Epoch {epoch}",
        disable=not coordinator.is_master(),
        total=num_steps_per_epoch,
        initial=start_step,
    ) as pbar:
        pbar_iter = iter(pbar)

        def fetch_data():
            step, batch = next(pbar_iter)
            pinned_video = batch["video"]
            batch["video"] = pinned_video.to(device, dtype, non_blocking=True)
            return batch, step, pinned_video

        batch_, step_, pinned_video_ = fetch_data()

        profiler_ctxt = (
            profile(
                activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
                schedule=my_schedule,
                on_trace_ready=torch.profiler.tensorboard_trace_handler("./log/profile"),
                record_shapes=True,
                profile_memory=True,
                with_stack=True,
            )
            if cfg.get("profile", False)
            else nullcontext()
        )

        with profiler_ctxt:
            for _ in range(start_step, num_steps_per_epoch):
                if cfg.get("profile", False) and _ == WARMUP + ACTIVE + WAIT + 3:
                    break

                # == load data ===
                batch, step, pinned_video = batch_, step_, pinned_video_
                if step + 1 < num_steps_per_epoch:
                    batch_, step_, pinned_video_ = fetch_data()

                # == log config ==
                global_step = epoch * num_steps_per_epoch + step
                actual_update_step = (global_step + 1) // accumulation_steps
                log_step += 1
                acc_step += 1

                # == mixed strategy ==
                x = batch["video"]
                t_length = x.size(2)
                use_video = 1
                if mixed_strategy == "mixed_video_image":
                    if random.random() < modulated_mixed_image_ratio and dist.get_rank() != 0:
                        # NOTE: enable the first rank to use video
                        t_length = 1
                        use_video = 0
                elif mixed_strategy == "mixed_video_random":
                    t_length = random.randint(1, x.size(2))
                x = x[:, :, :t_length, :, :]

                with Timer("model", log=True) if cfg.get("profile", False) else nullcontext():
                    # == forward pass ==
                    x_rec, posterior, z = model(x)

                    if cfg.get("profile", False):
                        profiler_ctxt.step()

                    if cache_pin_memory:
                        dataiter.remove_cache(pinned_video)

                    # == loss initialization ==
                    vae_loss = torch.tensor(0.0, device=device, dtype=dtype)
                    loss_dict = {}  # loss at every step

                    # == reconstruction loss ==
                    ret = vae_loss_fn(x, x_rec, posterior)
                    nll_loss = ret["nll_loss"]
                    kl_loss = ret["kl_loss"]
                    recon_loss = ret["recon_loss"]
                    perceptual_loss = ret["perceptual_loss"]
                    vae_loss += nll_loss + kl_loss

                    # == generator loss ==
                    if use_discriminator:
                        # turn off grad update for disc
                        discriminator.requires_grad_(False)
                        fake_logits = discriminator(x_rec.contiguous())

                        generator_loss, g_loss = generator_loss_fn(
                            fake_logits,
                            nll_loss,
                            model.module.get_last_layer(),
                            actual_update_step,
                            is_training=model.training,
                        )
                        # print(f"generator_loss: {generator_loss}, recon_loss: {recon_loss}, perceptual_loss: {perceptual_loss}")

                        vae_loss += generator_loss
                        # turn on disc training
                        discriminator.requires_grad_(True)

                    # == generator backward & update ==
                    ctx = (
                        booster.no_sync(model, optimizer)
                        if cfg.get("plugin", "zero2") in ("zero1", "zero1-seq")
                        and (step + 1) % accumulation_steps != 0
                        else nullcontext()
                    )
                    with Timer("backward", log=True) if cfg.get("profile", False) else nullcontext():
                        with ctx:
                            booster.backward(loss=vae_loss / accumulation_steps, optimizer=optimizer)

                    with Timer("optimizer", log=True) if cfg.get("profile", False) else nullcontext():
                        if (step + 1) % accumulation_steps == 0:
                            optimizer.step()
                            optimizer.zero_grad()
                            if lr_scheduler is not None:
                                lr_scheduler.step(
                                    actual_update_step,
                                )
                            # == update EMA ==
                            if ema is not None:
                                update_ema(
                                    ema,
                                    model.unwrap(),
                                    optimizer=optimizer,
                                    decay=cfg.get("ema_decay", 0.9999),
                                )

                # == logging ==
                log_loss("all", vae_loss, loss_dict, use_video)
                log_loss("nll", nll_loss, loss_dict, use_video)
                log_loss("nll_rec", recon_loss, loss_dict, use_video)
                log_loss("nll_per", perceptual_loss, loss_dict, use_video)
                log_loss("kl", kl_loss, loss_dict, use_video)
                if use_discriminator:
                    log_loss("gen_w", generator_loss, loss_dict, use_video)
                    log_loss("gen", g_loss, loss_dict, use_video)

                # == loss: discriminator adversarial ==
                if use_discriminator:
                    real_logits = discriminator(x.detach().contiguous())
                    fake_logits = discriminator(x_rec.detach().contiguous())
                    disc_loss = discriminator_loss_fn(
                        real_logits,
                        fake_logits,
                        actual_update_step,
                    )

                    # == discriminator backward & update ==
                    ctx = (
                        booster.no_sync(discriminator, disc_optimizer)
                        if cfg.get("plugin", "zero2") in ("zero1", "zero1-seq")
                        and (step + 1) % accumulation_steps != 0
                        else nullcontext()
                    )
                    with ctx:
                        booster.backward(loss=disc_loss / accumulation_steps, optimizer=disc_optimizer)
                    if (step + 1) % accumulation_steps == 0:
                        disc_optimizer.step()
                        disc_optimizer.zero_grad()
                        if disc_lr_scheduler is not None:
                            disc_lr_scheduler.step(actual_update_step)

                    # log
                    log_loss("disc", disc_loss, loss_dict, use_video)

                # == logging ==
                if (global_step + 1) % accumulation_steps == 0:
                    if coordinator.is_master() and actual_update_step % cfg.get("log_every", 1) == 0:
                        avg_loss = {k: v / log_step for k, v in running_loss.items()}
                        # progress bar
                        pbar.set_postfix(
                            {
                                # "step": step,
                                # "global_step": global_step,
                                # "actual_update_step": actual_update_step,
                                # "lr": optimizer.param_groups[0]["lr"],
                                **{k: f"{v:.2f}" for k, v in avg_loss.items()},
                            }
                        )
                        # tensorboard
                        tb_writer.add_scalar("loss", vae_loss.item(), actual_update_step)
                        # wandb
                        if cfg.get("wandb", False):
                            wandb.log(
                                {
                                    "iter": global_step,
                                    "epoch": epoch,
                                    "lr": optimizer.param_groups[0]["lr"],
                                    "avg_loss_": avg_loss,
                                    "avg_loss": avg_loss["all"],
                                    "loss_": loss_dict,
                                    "loss": vae_loss.item(),
                                    "global_grad_norm": optimizer.get_grad_norm(),
                                },
                                step=actual_update_step,
                            )

                        running_loss = {k: 0.0 for k in running_loss}
                        log_step = 0

                    # == checkpoint saving ==
                    ckpt_every = cfg.get("ckpt_every", 0)
                    if ckpt_every > 0 and actual_update_step % ckpt_every == 0 and coordinator.is_master():
                        subprocess.run("sudo drop_cache", shell=True)

                    if ckpt_every > 0 and actual_update_step % ckpt_every == 0:
                        # mannually garbage collection
                        gc.collect()

                        save_dir = checkpoint_io.save(
                            booster,
                            exp_dir,
                            model=model,
                            ema=ema,
                            optimizer=optimizer,
                            lr_scheduler=lr_scheduler,
                            sampler=sampler,
                            epoch=epoch,
                            step=step + 1,
                            global_step=global_step + 1,
                            batch_size=cfg.get("batch_size", None),
                            actual_update_step=actual_update_step,
                            ema_shape_dict=ema_shape_dict,
                            async_io=True,
                        )

                        if is_log_process(plugin_type, plugin_config):
                            os.system(f"chgrp -R share {save_dir}")

                        if use_discriminator:
                            booster.save_model(discriminator, os.path.join(save_dir, "discriminator"), shard=True)
                            booster.save_optimizer(
                                disc_optimizer,
                                os.path.join(save_dir, "disc_optimizer"),
                                shard=True,
                                size_per_shard=4096,
                            )
                            if disc_lr_scheduler is not None:
                                booster.save_lr_scheduler(
                                    disc_lr_scheduler, os.path.join(save_dir, "disc_lr_scheduler")
                                )
                        dist.barrier()

                        logger.info(
                            "Saved checkpoint at epoch %s, step %s, global_step %s to %s",
                            epoch,
                            step + 1,
                            actual_update_step,
                            save_dir,
                        )

                        # remove old checkpoints
                        rm_checkpoints(exp_dir, keep_n_latest=cfg.get("keep_n_latest", -1))
                        logger.info(
                            "Removed old checkpoints and kept %s latest ones.", cfg.get("keep_n_latest", -1)
                        )

        if cfg.get("profile", False):
            profiler_ctxt.export_chrome_trace("./log/profile/trace.json")

    sampler.reset()
    start_step = 0

    break